In [91]:
import pandas as pd # 0.23.0
import numpy as np

In [108]:
fileName = 'combinations.csv'
df = pd.read_csv(fileName)
df

,Template,#VIN,#VehicleCategoryCode,ClassOfHybridVehicleCode,#FuelTypeCode,#FuelCode1,#FuelCode2,#FuelCode3,FuelCode4,Info manueller Change in DB: vehicleNatureCode,Bemerkung
0,templates/Temp_NoHybrid_Bifuel.xml,BFE01emptyB1040$$,N1,Empty,B,10,40,-,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
1,templates/Temp_OVC_Bifuel.xml,BFE01vchevB10400$,N1,OVC-HEV,B,10,40,0,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
2,templates/Temp_NOVC_Bifuel.xml,BFE01nchevB10400$,N1,NOVC-HEV,B,10,40,0,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
3,templates/Temp_NoHybrid_Bifuel.xml,BFE38emptyB1040$$,N1,Empty,B,10,40,-,-,38,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
4,templates/Temp_OVC_Bifuel.xml,BFE38vchevB10400$,N1,OVC-HEV,B,10,40,0,-,38,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
...,...,...,...,...,...,...,...,...,...,...,...
58,templates/Temp_NOVC_Monofuel.xml,BFE01nchevM20$$0$,N1,NOVC-HEV,M,20,-,0,-,-,"Mono-Fuel, Diesel/ elektrisch, tstof F"
59,templates/Temp_OVC_Monofuel.xml,BFE38vchevM20$$0$,N1,OVC-HEV,M,20,-,0,-,38,"Mono-Fuel, Diesel/ elektrisch, tstof F"
60,templates/Temp_NOVC_Monofuel.xml,BFE38nchevM20$$0$,N1,NOVC-HEV,M,20,-,0,-,38,"Mono-Fuel, Diesel/ elektrisch, tstof F"
61,templates/Temp_OVC_Monofuel.xml,BFE30vchevM20$$0$,M1,OVC-HEV,M,20,-,0,-,-,"Mono-Fuel, Diesel/ elektrisch, tstof F"


In [109]:
df.columns

Index(['Template', '#VIN', '#VehicleCategoryCode', 'ClassOfHybridVehicleCode',
       '#FuelTypeCode', '#FuelCode1', '#FuelCode2', '#FuelCode3', 'FuelCode4',
       'Info manueller Change in DB: vehicleNatureCode ', 'Bemerkung'],
      dtype='object')

In [110]:
cols = ['#VehicleCategoryCode', 'ClassOfHybridVehicleCode',
        '#FuelTypeCode', '#FuelCode1', '#FuelCode2', '#FuelCode3',
        'FuelCode4', 'Info manueller Change in DB: vehicleNatureCode ']

allLists = []

for col in cols:
  allLists.append(df[col].unique())

len(allLists)
allLists[7] = ['-', '35', '38']

In [111]:
newDF = pd.DataFrame(columns=cols)

for a in allLists[0]:
  for b in allLists[1]:
    for c in allLists[2]:
      for d in allLists[3]:
        for e in allLists[4]:
          for f in allLists[5]:
            for g in allLists[6]:
              for h in allLists[7]:

                car = [a, b, c, d, e, f, g, h]
                newDF.loc[len(newDF)] = car

In [112]:
newDF['#VIN'] = '-'
newDF = newDF[['#VIN', '#VehicleCategoryCode', 'ClassOfHybridVehicleCode',
               '#FuelTypeCode', '#FuelCode1', '#FuelCode2', '#FuelCode3',
               'FuelCode4', 'Info manueller Change in DB: vehicleNatureCode ']]

In [113]:
# map the following values
dict_VCC = {'N1': '01', 'M1': '30'} # 35 and 38 need to be mapped
dict_CHVC = {'Empty': 'empty', 'OVC-HEV': 'vchev',
             'NOVC-HEV': 'nchev', 'OVC-FCHV': 'cfchv',
             'NOVC-FCHV': 'nfchv'}


def createVIN(row):

  emptyElement = '$'

  newVIN = 'BFE'

  # check for N1
  vcc = row['#VehicleCategoryCode']

  if vcc == 'N1':

    com = row['Info manueller Change in DB: vehicleNatureCode ']

    if com  != '-':
      newVIN = newVIN + com
    else:
      newVIN = newVIN + dict_VCC[vcc]
  else:
    newVIN = newVIN + dict_VCC[vcc]


  newVIN = newVIN + dict_CHVC[row['ClassOfHybridVehicleCode']]


  fuelTypeCode = row['#FuelTypeCode']

  if fuelTypeCode == '-':
    newVIN = newVIN + emptyElement
  else:
    newVIN = newVIN + fuelTypeCode

  for col in ['#FuelCode1','#FuelCode2', '#FuelCode3']:

    val = str(row[col])

    if len(val) < 2:
      if val == '-':
        val = emptyElement + emptyElement
      else:
        val = val + emptyElement

    newVIN = newVIN + val

  return newVIN

In [114]:
newDF['#VIN'] = newDF.apply(lambda row: createVIN(row), axis=1)

In [115]:
OVCtemplates = {'NOVC-HEV':  ['templates/Temp_NOVC_Monofuel.xml',
                              'templates/Temp_NOVC_Bifuel.xml',
                              'templates/Temp_NOVC_Bifuel.xml'],
                'NOVC-FCHV': ['templates/Temp_NOVC-FCHV.xml'],
                'OVC-HEV':   ['templates/Temp_OVC_Monofuel.xml',
                              'templates/Temp_OVC_Bifuel.xml',
                              'templates/Temp_OVC_Trifuel.xml'],
                'OVC-FCHV':  ['templates/Temp_OVC-FCHV.xml']}

noHybrid = ['templates/Temp_NoHybrid_Monofuel.xml',
            'templates/Temp_NoHybrid_Bifuel.xml',
            'templates/Temp_NoHybrid_Trifuel.xml']

In [116]:
fuelCodes = ['#FuelCode1', '#FuelCode2', '#FuelCode3']

def chooseTemplate(row):
  fuel = 0

  # to check if it's mono-, bi- or trifuel
  for fuelCode in fuelCodes:

    if row[fuelCode] != '-' and row[fuelCode] != 0:
      fuel += 1

  cohvc = row['ClassOfHybridVehicleCode']

  if 'OVC' in cohvc:
    possibleTemplates = OVCtemplates[cohvc]

    if len(possibleTemplates) > 1:
      if fuel == 0:
        return '!'
      else:
        return possibleTemplates[fuel-1]
    else:
      return possibleTemplates[0]

  else:
    #if fuel == 1 and row['#FuelCode1'] == 50:

    fuelCode1 = row['#FuelCode1']

    #if fuel == 1 and row['#FuelCode1'] == 50:
    if fuelCode1 == 50:
      return 'templates/Temp_Temp_WASSERSTOFF_W.xml'

    elif fuelCode1 == 0:
      return 'templates/Temp_Temp_WASSERSTOFF_W.xml'

    else:
      if fuel == 0:
        return '!'
      else:
        return noHybrid[fuel-1]


In [117]:
newDF['Template'] = '-'
newDF = newDF[['Template', '#VIN', '#VehicleCategoryCode', 'ClassOfHybridVehicleCode',
               '#FuelTypeCode', '#FuelCode1', '#FuelCode2', '#FuelCode3',
               'FuelCode4', 'Info manueller Change in DB: vehicleNatureCode ']]
newDF

,Template,#VIN,#VehicleCategoryCode,ClassOfHybridVehicleCode,#FuelTypeCode,#FuelCode1,#FuelCode2,#FuelCode3,FuelCode4,Info manueller Change in DB: vehicleNatureCode
0,-,BFE01emptyB1040$$,N1,Empty,B,10,40,-,-,-
1,-,BFE35emptyB1040$$,N1,Empty,B,10,40,-,-,35
2,-,BFE38emptyB1040$$,N1,Empty,B,10,40,-,-,38
3,-,BFE01emptyB1040$$,N1,Empty,B,10,40,-,0,-
4,-,BFE35emptyB1040$$,N1,Empty,B,10,40,-,0,35
...,...,...,...,...,...,...,...,...,...,...
8995,-,BFE30nfchvD20$$44,M1,NOVC-FCHV,D,20,-,44,-,35
8996,-,BFE30nfchvD20$$44,M1,NOVC-FCHV,D,20,-,44,-,38
8997,-,BFE30nfchvD20$$44,M1,NOVC-FCHV,D,20,-,44,0,-
8998,-,BFE30nfchvD20$$44,M1,NOVC-FCHV,D,20,-,44,0,35


In [118]:
newDF['Template'] = newDF.apply(lambda row: chooseTemplate(row), axis=1)

In [119]:
indx = newDF[(newDF['#VehicleCategoryCode']=='M1') & (newDF['Info manueller Change in DB: vehicleNatureCode ']!='-')].index
newDF.drop(indx, inplace=True)

In [120]:
newDF

,Template,#VIN,#VehicleCategoryCode,ClassOfHybridVehicleCode,#FuelTypeCode,#FuelCode1,#FuelCode2,#FuelCode3,FuelCode4,Info manueller Change in DB: vehicleNatureCode
0,templates/Temp_NoHybrid_Bifuel.xml,BFE01emptyB1040$$,N1,Empty,B,10,40,-,-,-
1,templates/Temp_NoHybrid_Bifuel.xml,BFE35emptyB1040$$,N1,Empty,B,10,40,-,-,35
2,templates/Temp_NoHybrid_Bifuel.xml,BFE38emptyB1040$$,N1,Empty,B,10,40,-,-,38
3,templates/Temp_NoHybrid_Bifuel.xml,BFE01emptyB1040$$,N1,Empty,B,10,40,-,0,-
4,templates/Temp_NoHybrid_Bifuel.xml,BFE35emptyB1040$$,N1,Empty,B,10,40,-,0,35
...,...,...,...,...,...,...,...,...,...,...
8985,templates/Temp_NOVC-FCHV.xml,BFE30nfchvD20$$$$,M1,NOVC-FCHV,D,20,-,-,0,-
8988,templates/Temp_NOVC-FCHV.xml,BFE30nfchvD20$$0$,M1,NOVC-FCHV,D,20,-,0,-,-
8991,templates/Temp_NOVC-FCHV.xml,BFE30nfchvD20$$0$,M1,NOVC-FCHV,D,20,-,0,0,-
8994,templates/Temp_NOVC-FCHV.xml,BFE30nfchvD20$$44,M1,NOVC-FCHV,D,20,-,44,-,-


In [121]:
newDF.to_csv('allPossibleCombinations.csv', index=False)